In [1]:
import add_packages
import os
from pprint import pprint
import pandas as pd
from operator import itemgetter

from toolkit import sql
from toolkit.langchain import (
	text_embedding_models, stores, prompts, output_parsers, agents, runnables,
	chains, models, tools, graphs
)

In [3]:
# my_sql_db = sql.MySQLDatabase()
my_sql_db = sql.MySQLDatabase(
	dbname=os.getenv("SQL_DB_NEON"),
	host=os.getenv("SQL_HOST_NEON"),
	port=os.getenv("SQL_PORT_NEON"),
	user=os.getenv("SQL_USER_NEON"),
	password=os.getenv("SQL_PASSWORD_NEON"),
)

my_table_schema = [
	"id SERIAL",
	"name VARCHAR(50) NOT NULL UNIQUE",
	"type VARCHAR(20) NOT NULL",
	"engine_type VARCHAR(20) NOT NULL",
	"webpage_link VARCHAR(200) NOT NULL",
	"specs TEXT NOT NULL",
	"brochure_link VARCHAR(200) NULL",
	"features TEXT NOT NULL",
	"interior_features TEXT NOT NULL",
	"exterior_features TEXT NOT NULL",
	"colors VARCHAR(200) NOT NULL",
	"price TEXT NOT NULL",
	"PRIMARY KEY (id)",
]
my_table = sql.MySQLTable(
	name="vinfast_cars", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS


In [4]:
path_xlsx = f"{add_packages.APP_PATH}/data/vinfast/cars.xlsx"
df = pd.read_excel(path_xlsx)

# my_table.insert_from_dataframe(df)


In [ ]:
table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]
df.columns = table_cols
# my_table.insert_from_dataframe(df)

# INSERT DATA TO TABLE
cols = ["name", "type", "engine_type"]
proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
questions = [
	"How many cars are there?",	
	"Could you tell me the different types of vehicles included in this database?",
	"What are the price ranges for the various vehicle models?",
	"Are there any electric vehicles in the database, and if so, what are their key specifications like range and charging time?",
	"Which models offer different color options, and what are those color choices?",
	"Can you provide details on the interior and exterior features available for the different vehicle models?",
	"Are there any brochures or links available that provide more comprehensive information about the vehicles?",
	"How does the vehicle pricing work for models that offer battery rental or purchase options?",
	"Are there any performance-related specifications like engine power, torque, and acceleration times listed for the vehicles?",
	"Does the database include information on vehicle dimensions or seating capacity?",
	"Can you give me an overview of the advanced safety features or driver assistance technologies offered in these vehicles?",
]

In [ ]:
examples_questions_to_sql = [
    {
        "input": "How many cars are there?",
        "query": "SELECT COUNT(*) FROM vinfast_cars;"
    },
    {
        "input": "Could you tell me the different types of vehicles included in this database?",
        "query": "SELECT DISTINCT type FROM vinfast_cars;"
    },
    {
        "input": "What are the price ranges for the various vehicle models?",
        "query": "SELECT name, price FROM vinfast_cars;"
    },
    {
        "input": "Are there any electric vehicles in the database, and if so, what are their key specifications like range and charging time?",
        "query": "SELECT name, specs FROM vinfast_cars WHERE engine_type = 'Điện';"
    },
    {
        "input": "Which models offer different color options, and what are those color choices?",
        "query": "SELECT name, colors FROM vinfast_cars WHERE colors <> '';"
    },
    {
        "input": "Can you provide details on the interior and exterior features available for the different vehicle models?",
        "query": "SELECT name, interior_features, exterior_features FROM vinfast_cars;"
    },
    {
        "input": "Are there any brochures or links available that provide more comprehensive information about the vehicles?",
        "query": "SELECT name, brochure_link, webpage_link FROM vinfast_cars WHERE brochure_link <> '' OR webpage_link <> '';"
    },
    {
        "input": "How does the vehicle pricing work for models that offer battery rental or purchase options?",
        "query": "SELECT name, price FROM vinfast_cars WHERE price LIKE '%PIN%';"
    },
    {
        "input": "Are there any performance-related specifications like engine power, torque, and acceleration times listed for the vehicles?",
        "query": "SELECT name, specs FROM vinfast_cars WHERE specs LIKE '%Công suất%' OR specs LIKE '%Mô men xoắn%' OR specs LIKE '%Tăng tốc%';"
    },
    {
        "input": "Does the database include information on vehicle dimensions or seating capacity?",
        "query": "SELECT name, specs FROM vinfast_cars WHERE specs LIKE '%Kích thước%' OR specs LIKE '%Số ghế ngồi%';"
    },
    {
        "input": "Can you give me an overview of the advanced safety features or driver assistance technologies offered in these vehicles?",
        "query": "SELECT name, features FROM vinfast_cars WHERE features LIKE '%Cảnh báo%' OR features LIKE '%Hỗ trợ%';"
    }
]

# Data

In [ ]:
schema_vinfast_cars = [
	"id SERIAL",
	"name VARCHAR(50) NOT NULL UNIQUE",
	"type VARCHAR(20) NOT NULL",
	"engine_type VARCHAR(20) NOT NULL",
	"webpage_link VARCHAR(200) NOT NULL",
	"specs TEXT NOT NULL",
	"brochure_link VARCHAR(200) NULL",
	"features TEXT NOT NULL",
	"interior_features TEXT NOT NULL",
	"exterior_features TEXT NOT NULL",
	"colors VARCHAR(200) NOT NULL",
	"price TEXT NOT NULL",
	"PRIMARY KEY (id)",
]

table_vinfast_cars = sql.MySQLTable(
	name="vinfast_cars", 
	schema=schema_vinfast_cars,
	db=my_sql_db,
)

table_vinfast_cars.create()

In [ ]:
table_vinfast_cars.insert_from_excel(f"{add_packages.APP_PATH}/data/vinfast/cars.xlsx")

# Docs

## [Usecase - Build a Question/Answering system over SQL data](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/)

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

### Chains


In [ ]:
chain = chains.create_sql_query_chain(llm, db)

In [ ]:
def query_processor(query: str):
	query = query.replace("\\", "")
	query = query.replace("SQLQuery: ", "")
	return query

In [ ]:
response = chain.invoke({"question": "How many cars are there?"})
response = query_processor(response)

In [ ]:
db.run(response)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
query_writer = chains.create_sql_query_chain(llm, db)
query_executor = tools.QuerySQLDataBaseTool(db=db)

prompt_answer = prompts.PromptTemplate.from_template("""\
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:\
""")

chain = (
	runnables.RunnablePassthrough.assign(query=query_writer).assign(
		result=itemgetter("query") 
					| runnables.RunnableLambda(query_processor) 
					| query_executor
	)
	| prompt_answer
	| llm
	| output_parsers.StrOutputParser()
)

In [ ]:
chain.invoke({"question": "How many cars are there?"})

### Agents


In [ ]:
sql_toolkit = tools.SQLDatabaseToolkit(db=db, llm=llm)
sql_tools = sql_toolkit.get_tools()

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = prompts.SystemMessage(content=SQL_PREFIX)

agent_executor = graphs.chat_agent_executor.create_tool_calling_executor(
	llm, sql_tools, messages_modifier=system_message
)

In [ ]:
result = agent_executor.invoke(
	{"messages": [
		prompts.HumanMessage(
			content=questions[0]
	)
	]}
)
result = dict(list(result.items())[0][-1][-1])["content"]

pprint(result)

## Components

### Tools

#### [SQL Database](https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/)


### Toolkits

#### [SQL Database](https://python.langchain.com/v0.2/docs/integrations/toolkits/sql_database/)


In [ ]:
# More ok

agent_executor = agents.create_sql_agent(
	llm=llm, db=db, agent_type="tool-calling", verbose=True,
)


In [ ]:
agent_executor.invoke(
	# questions[0]
	# "tell me about vf3"
	# "colors of vf3"
	"color difference between vf3 and vf7"
)

In [ ]:
sql_toolkit = tools.SQLDatabaseToolkit(db=db, llm=llm)
sql_context = sql_toolkit.get_context()
sql_tools = sql_toolkit.get_tools()

messages = [
	prompts.HumanMessagePromptTemplate.from_template("{input}"),
	prompts.AIMessage(content=prompts.SQL_FUNCTIONS_SUFFIX),
	prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
]
prompt = prompts.ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**sql_context)

agent = agents.create_tool_calling_agent(
	llm=llm, tools=sql_tools, prompt=prompt,
)
agent_executor = agents.AgentExecutor(
	agent=agent,
	tools=sql_tools,
	verbose=True,
)

In [ ]:
agent_executor.invoke({
	"input": questions[0]
})

## How-to

### [better prompt](https://python.langchain.com/v0.2/docs/how_to/sql_prompting/)

In [ ]:
# Dialect-specific prompting
dialects = list(prompts.SQL_PROMPTS)

chain = chains.create_sql_query_chain(llm, db)
prompt_chain = chain.get_prompts()[0]
# prompt_chain.pretty_print()

context = db.get_context()
# print(f"contexts: {list(context)}\n")
# print(f"table_info: {context['table_info']}")

prompt_with_context = prompt_chain.partial(table_info=context['table_info'])
# print(prompt_with_context.pretty_repr())

example_selector = prompts.SemanticSimilarityExampleSelector.from_examples(
	examples=examples_questions_to_sql,
	embeddings=embeddings,
	vectorstore_cls=vectorstore,
	k=5,
	input_keys=["input"],
)
# example_selector.select_examples({
# 	"input": questions[0]
# })

prompt_tpl_fewshot = prompts.PromptTemplate.from_template("""\
User input: {input}
SQL query: {query}\
""")
prompt_fewshot = prompts.FewShotPromptTemplate(
	# examples=examples_questions_to_sql, # option
	example_selector=example_selector, # option
	example_prompt=prompt_tpl_fewshot,
	prefix="You are a SQL expert. Given an input question, create a syntactically correct SQL query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
	suffix="User input: {input}\nSQL query: ",
	input_variables=["input", "top_k", "table_info"],
)
# print(prompt_fewshot.format(input="How many cars are there?", top_k=3, table_info="foo"))

chain = chains.create_sql_query_chain(llm, db, prompt_fewshot)

In [ ]:
chain.invoke({
	"question": questions[0]
})

### [do query validation](https://python.langchain.com/v0.2/docs/how_to/sql_check_querying/)


In [ ]:

prompt_system_check_query = """\
You are a {dialect} expert. Given an input question, creat a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>\
"""
prompt_check_query = prompts.ChatPromptTemplate.from_messages([
	("system", prompt_system_check_query),
	("human", "{input}"),
]).partial(dialect=db.dialect)

def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]

chain_gen_check_query = (
	chains.create_sql_query_chain(llm, db, prompt=prompt_check_query)
	| parse_final_answer
)

query_executor = tools.QuerySQLDataBaseTool(db=db)

prompt_answer = prompts.PromptTemplate.from_template("""\
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:\
""")

chain_check_query = (
  runnables.RunnablePassthrough
  .assign(query=chain_gen_check_query)
  .assign(result=itemgetter("query") | query_executor)
	| prompt_answer
	| llm
	| output_parsers.StrOutputParser()
).with_retry()

In [ ]:
chain_check_query.invoke({
	"question": questions[0]
})


### [deal with large databases](https://python.langchain.com/v0.2/docs/how_to/sql_large_db/)

#### Many tables


In [ ]:
categories = ["car", "dog"]


In [ ]:
# class Table(tools.BaseModel):
# 	"""Table in SQL database."""

# 	name: str = tools.Field(description="Name of table in SQL database.")

# prompt_tpl_get_tables = f"""\
# Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
# The tables are:

# {table_names}

# Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.\
# """

# prompt_get_tables = prompts.ChatPromptTemplate.from_messages([
# 	("system", prompt_tpl_get_tables),
# 	("human", "{input}"),
# ])
# llm_with_tools = llm.bind_tools([Table])

# def chain_process_get_tables(tables: list):
# 	result = [table.name for table in tables]
# 	return result

# chain_get_tables = (
#   {
# 		"input": itemgetter("question")
# 	}
#   |	prompt_get_tables
# 	| llm_with_tools
# 	| tools.PydanticToolsParser(tools=[Table])
# 	| runnables.RunnableLambda(chain_process_get_tables)
# )

# chain_check_table_gen_check_query = (
#   runnables.RunnablePassthrough
#   .assign(table_names_to_use=chain_get_tables)
#   .assign(query=chain_gen_check_query)
#   .assign(result=itemgetter("query") | query_executor)
# 	| prompt_answer
# 	| llm
# 	| output_parsers.StrOutputParser()
# ).with_retry()

In [ ]:
# result = chain_check_table_gen_check_query.invoke({
# 	"question": questions[0]
# 	# "question": "tell me about VF 3"
 
# })

# pprint(result)


#### High-cardinality columns

In [ ]:
# prompt_tpl_write_sql_filter_nouns = """\
# You are a SQL expert. Given an input question, create a syntactically
# correct SQL query to run. Unless otherwise specificed, do not return more than
# {top_k} rows.

# Only return the SQL query with no markup or explanation.

# Here is the relevant table info: {table_info}

# Here is a non-exhaustive list of possible feature values. If filtering on a feature
# value make sure to check its spelling against this list first:

# {proper_nouns}\
# """

# prompt_write_sql_filter_nouns = prompts.ChatPromptTemplate.from_messages([
# 	("system", prompt_tpl_write_sql_filter_nouns),
# 	("human", "{input}"),
# ])
# chain_write_sql = chains.create_sql_query_chain(
#   llm, db, prompt=prompt_write_sql_filter_nouns
# )

# chain_retrieve_proper_nouns = (
# 	itemgetter("question")
# 	| retriever_nouns
# 	| (lambda docs: "\n".join(doc.page_content for doc in docs))
# )

# chain_write_sql_filter_nouns = runnables.RunnablePassthrough\
# 	.assign(proper_nouns=chain_retrieve_proper_nouns)\
# 	| chain_write_sql

In [ ]:
# query = chain_write_sql_filter_nouns.invoke({
# 	"question": questions[0]
# })

# query

### [do question answering over CSVs](https://python.langchain.com/v0.2/docs/how_to/sql_csv/)

## Cookbook

### [LLaMA2 chat with SQL](https://github.com/langchain-ai/langchain/blob/master/cookbook/LLaMA2_sql_chat.ipynb)

### [Databricks](https://github.com/langchain-ai/langchain/blob/master/cookbook/databricks_sql_db.ipynb)

### [Vector SQL Retriever with MyScale](https://github.com/langchain-ai/langchain/blob/master/cookbook/myscale_vector_sql.ipynb)

### [Incoporating semantic similarity in tabular databases](https://github.com/langchain-ai/langchain/blob/master/cookbook/retrieval_in_sql.ipynb)


### [SQL Database Chain](https://github.com/langchain-ai/langchain/blob/master/cookbook/sql_db_qa.mdx)


# Custom

## Chain

In [ ]:
class Table(tools.BaseModel):
	"""Table in SQL database."""

	name: str = tools.Field(description="Name of table in SQL database.")

prompt_tpl_get_tables = f"""\
Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.\
"""

prompt_get_tables = prompts.ChatPromptTemplate.from_messages([
	("system", prompt_tpl_get_tables),
	("human", "{input}"),
])
llm_with_tools = llm.bind_tools([Table])

def chain_process_get_tables(tables: list):
	result = [table.name for table in tables]
	return result

chain_get_tables = (
  {
		"input": itemgetter("question")
	}
  |	prompt_get_tables
	| llm_with_tools
	| tools.PydanticToolsParser(tools=[Table])
	| runnables.RunnableLambda(chain_process_get_tables)
)
#*-----------------------------------------------------------------------------

chain_retrieve_proper_nouns = (
	itemgetter("question")
	| retriever_nouns
	| (lambda docs: "\n".join(doc.page_content for doc in docs))
)

#*-----------------------------------------------------------------------------
example_selector = prompts.SemanticSimilarityExampleSelector.from_examples(
	examples=examples_questions_to_sql,
	embeddings=embeddings,
	vectorstore_cls=vectorstore,
	k=5,
	input_keys=["input"],
)

chain_get_examples = (
	{
		"input": itemgetter("question")
	}
	| runnables.RunnableLambda(example_selector.select_examples)
)

#*-----------------------------------------------------------------------------
# Gen sql, check sql, filter nouns
prompt_tpl_write_sql = """\
You are a {dialect} expert. Given an input question, creat a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Here is the relevant table info:
{table_info}

Here is a non-exhaustive list of possible feature values. If filtering on a feature
value make sure to check its spelling against this list first:
{proper_nouns}

Below are a number of examples of questions and their corresponding SQL queries.
{examples}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>\
"""
prompt_write_sql = prompts.ChatPromptTemplate.from_messages([
	("system", prompt_tpl_write_sql),
	("human", "{input}"),
]).partial(dialect=db.dialect)

def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]
  
chain_write_sql = (
	chains.create_sql_query_chain(
		llm, db, prompt=prompt_write_sql
	)
	| runnables.RunnableLambda(parse_final_answer)	
)

#*-----------------------------------------------------------------------------

query_executor = tools.QuerySQLDataBaseTool(db=db)

prompt_answer = prompts.PromptTemplate.from_template("""\
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:\
""")

chain_answer = prompt_answer | llm | output_parsers.StrOutputParser()

chain_sql = (
  runnables.RunnablePassthrough
  .assign(table_names_to_use=chain_get_tables)
  .assign(proper_nouns=chain_retrieve_proper_nouns)
  .assign(examples=chain_get_examples)
  .assign(query=chain_write_sql)
  .assign(result=itemgetter("query") | query_executor)
	.assign(output=chain_answer)
).with_retry()


In [ ]:
result = chain_sql.invoke({
	# "question": questions[0]
	# "question": "tell me about vf3"
	# "question": "colors of vf3"
	"question": "color difference between vf3 and vf7"
})
output = result["output"]

pprint(output)

### Class

In [ ]:
my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	is_debug=True,
	tool_name="sql_executor",
	tool_description="Generate SQL based on user question and execute it",
	tool_metadata={"data": "cars"},
	tool_tags=["cars"],
)

In [ ]:
result = my_sql_chain.invoke_chain(
	questions[0]
	# "tell me about vf3"
	# "color difference between vf3 and vf7"
)

pprint(result)

In [ ]:
tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [ ]:
llm = models.chat_openai
my_tools = [
	tool_chain_sql,
]
prompt = prompts.create_prompt_tool_calling_agent()

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=my_tools,
	prompt=prompt,
	agent_type='tool_calling',
	agent_verbose=True,
)

In [ ]:
await agent.invoke_agent(
	# input_message=questions[0],
	input_message="tell me about vf3",
	history_type="in_memory",
	user_id="user",
	session_id="default",
)

## Agent

# Todos

## Youtube

- [ ] https://alejandro-ao.com/chat-with-mysql-using-python-and-langchain/
- [ ] https://github.com/alejandro-ao/chat-with-mysql
- [ ] https://github.com/Coding-Crashkurse/Langchain-Dynamic-Routing
- [ ] https://python.langchain.com/v0.1/docs/integrations/toolkits/sql_database/
- [ ] https://blog.futuresmart.ai/mastering-natural-language-to-sql-with-langchain-nl2sql
- [ ] https://github.com/krishnaik06/Google-Gemini-Crash-Course/tree/main/sqlllm
- [ ] https://github.com/langchain-ai/langchain/tree/master/templates/sql-research-assistant
- [ ] https://www.rabbitmetrics.com/building-llm-knowledge-base-for-advanced-sql-chains/
- [ ] https://www.rabbitmetrics.com/chatting-with-ecommerce-data/
- [ ] https://github.com/Farzad-R/Advanced-QA-and-RAG-Series
- [ ] https://www.youtube.com/watch?v=XNeTgVEzILg (No code)
---
- [ ] https://www.youtube.com/watch?v=es-9MgxB-uc

## Docs
---
- [ ] Not Done
- [x] Done

### Tools